# Twitterでフォロワーをクラスタリングしてみた。
---
分析

## 目次
- フォロワー同士の関係を取得
- グラフの定義・クラスタ

In [3]:
import os, sys
sys.path.append('..')

from tqdm import tqdm
import psycopg2

import pandas as pd
import numpy as np

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

## 1. フォロワー同士の関係を取得

In [5]:
# 接続情報
connection_config = {
    'host': 'localhost',
    'port': '5432',
    'database': 'my_db',
    'user': 'rui',
    'password': 'password'
}

conn = psycopg2.connect(**connection_config)

In [30]:
q_screen_name = 'rui_308'
q = "SELECT * FROM twitter.follower_user_info where slave_screen_name='%s'" % q_screen_name

follower_df = pd.read_sql(sql=q, con=conn)

In [33]:
follower_df.head()

,slave_screen_name,user_id,screen_name,friends_count,followers_count,self_discription,protected
0,rui_308,886745335,classics_jp,3074,2814,"ホメロス(H),アイスキュロス(Ac),ソフォクレス(S),エウリピデス(E),アリストパネ...",0
1,rui_308,505760483,DM_Hero,4852,3806,最近のデュエマで勝手に思ったことをカード別につぶやくbotです デュエルマスターズをやってる...,0
2,rui_308,379144705,ayataka_bot,61,52,http://t.co/OGdrONxkEE←説明書 創作bot。高天原防衛軍二番隊隊長の一...,0
3,rui_308,312027457,AAtest_bot,345,267,2chはモナー板、長編AA板の名言・迷言などを幅広く集めていくつもりのbot。こちらはテスト...,0
4,rui_308,2944058462,GMAhikaru,2500,2911,Lv17電波少女 邦ロック,0


In [43]:
q_follower_ids = follower_df['user_id']

In [69]:
q = "select * from twitter.friends"

friends_df = pd.read_sql(sql=q, con=conn)

In [70]:
friends_df['com_friend_ids'] = friends_df['friend_ids'].str.split(',').apply(lambda x: list(set(x) & set(q_follower_ids)))

In [71]:
friends_df.head()

,user_id,screen_name,friend_ids,com_friend_ids
0,1907355950,0127jdabcyasu,"1012930315755507713,946948515514609665,3145665...",[]
1,800313087039410177,ONEOKROCK_Mamo,"776049295,2939946835,907526713,819951672063447...",[]
2,777882633950801920,live_love_333,"239782959,843863742504886272,89387209332177305...",[]
3,760426981594771456,s_ALXD,"3145665278,4431646093,976452610596126723,90800...",[]
4,2662271406,hinatanu0713,"883850801889689600,883644726846439424,10107728...",[]


In [72]:
friends_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 4 columns):
user_id           370 non-null int64
screen_name       370 non-null object
friend_ids        370 non-null object
com_friend_ids    370 non-null object
dtypes: int64(1), object(3)
memory usage: 11.6+ KB


In [73]:
len(list(set(friends_df['user_id']) & set(q_follower_ids)))

370

In [74]:
friends_df['com_friend_ids'].apply(lambda x: len(x)).sum()

0